# Heurística QKP $0$-$1$

## Dados

Considere dos seguintes dados:

-  $N = \{1, \ldots, n\}$ um conjunto dos objetos.

-  $P = \{p_{ij} \in \mathbb{R}_{+} : \ i \leq j, \ i, j \in N \}$ o conjunto dos benefícios aos itens de $N$ e a relação entre eles.

-  $W = \{ w_i \in \mathbb{N} : i \in N \}$ o conjunto dos pesos dos itens de $N$.

-  $c \in \mathbb{N}$ capacidade da mochila, onde $\displaystyle\max_{i \in N} w_i \leq c < \sum_{i  \in N} w_i$.

## QKP 0-1

Utilizando os dados acima para definir o problema da mochila quadrática 0-1.

\begin{align}
max \; & \sum_{i \in N} q_{i} x_i + \sum_{i = 1}^{n-1} \sum_{j =i+1}^{n}
p_{ij} x_{i} x_{j} \\
s. a  \; & \sum_{i \in N} w_{i} x_{i} \leq c, \\
& x_{i} \in \left\{0, 1 \right\}, \ \ i  \in N.
\end{align}

onde $q_{i} = p_{ii}, \ i \in N$.

### Um limite inferiore para o QKP

Limites inferiores podem ser obtidos de modo eficiente através de heurísticas. 

Billionnet e Calmels(1996) propuseram uma **heurística** para gerar um limite inferior para o QKP, onde primeiro geram uma solução gulosa inicial definindo $x_{j} = 1$ para todo $j \in N$, e então interativamente trocam o valor das variáveis de $1$ para $0$, de modo a atingir o menor decréscimo na função objetivo, até que uma solução viável é atingida.

Em seguida um procedimento denominado **melhora** é executado, onde uma sequência de iterações são executadas com a finalidade de melhorar a solução através de busca local. 

Seja 
$$S = \left\{ j \in N : \ x_{j} = 1 \right\}$$
o conjunto que define a atual solução.

Para cada $j \in N \setminus S$, se 
$$w_{j} + \displaystyle\sum_{l\in S} w_{l} \leq c$$ 
defina $I_{j} = \emptyset$ e seja $\delta_{j}$ o valor que define o crescimento da função objetivo quando $x_{j}$ toma valor 1, caso contrário, seja $\delta_{j}$ o maior crescimento quando definimos $x_{j} = 1$ e $x_{i} = 0$ para algum $i \in S$ tal que 
$$w_{j} - w_{i} + \displaystyle\sum_{l \in S} w_l \leq c$$ 
e seja $I_{j} = \left\{ i \right\}$

Escolha um $k$ tal que 
$$\delta_{k} = \displaystyle\max_{j \in   N  \setminus S} \delta_{j}$$

A heurística finaliza se $\delta_{k} \leq 0$, caso contrário o corrente conjunto solução é definido como $S \setminus I_{k} \cup \left\{ k \right\}$ e outra iteração é executada.


## Implementação da heuristica

In [55]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [56]:
def read_instance(dim,perc,t):

    instance = f"instances/{dim}/{dim}_{perc}_{t}.txt"

    with open(instance, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # ler o tamanho da instancia
    n = int(linhas[0]) 

    # ler a diagonal da matriz
    d = np.fromstring(linhas[1], dtype=int, sep = ' ') 

    # define a matriz
    p = np.zeros((n,n), dtype=int) 

    # preenche a diagonal
    for i in range(n): 
        p[i][i] = d[i]

    # preenche o resto da matriz
    for i in range(n-1): 
        linha = np.fromstring(linhas[i+2], dtype=int, sep = ' ')
        for j in range(n-(i+1)):
            p[i][j+i+1] = linha[j]
            p[j+i+1][i] = p[i][j+i+1]

    # ler a capacidade
    c = int(linhas[n+2]) 

    # ler os pesos
    w = np.fromstring(linhas[n+3], dtype=int, sep = ' ') 

    return n, p, w, c

In [57]:
def guloso(n, p, w, c):
    psum = wsum = 0
  
    x = [0 for i in range(n)]
  
    for i in range(n):
        wsum += w[i]
        for j in range(n):
            psum += p[i][j]
  
    ptot = psum
  
    for i in range(n):
        x[i] = 1
  
    while True:
        mineff = ptot
        mini = -1
        for  i in range(n):
            if x[i] == 0:
                continue
            pi = -p[i][i]
            for j in range(n):
                if x[j] != 0:
                    pi += p[j][i] + p[i][j]

            eff = pi / w[i]
            if eff < mineff:
                mineff = eff
                mini = i
                minp = pi
        if mini == -1:
            print("error\n")
            exit
        i = mini
        x[i] = 0
        psum -= minp
        wsum -= w[i]
        if wsum <= c:
            break

    #lb = psum
    lb = 0
    for i in range(n):
        if x[i] != 0: 
            lb += p[i][i]
            for j in range(i+1,n):
                if x[j] != 0:
                    lb += p[i][j]
  
    return lb, x

In [58]:
def melhora(n, p, w, c, xprime, lb):

    xstar = [0 for i in range(n)]

    res = c
  
    q = [0 for i in range(n)]
  
    for i in range(n):
        if xprime[i] != 0:
            res -= w[i]
  
    while True:
        for i in range(n):
            tot = p[i][i]
            for j in range(n):
                if (j != i) and (xprime[j] != 0):
                    tot += p[i][j] + p[j][i]
            q[i] = tot
    
        bgain = gaini = gainj = 0
    
        for i in range(n):
            if xprime[i] == 0:
                if w[i] <= res:
                    gain = q[i]
                    if gain > bgain:
                        bgain = gain
                        gaini = i
                        gainj = -1
                else:
                    for j in range(n):
                        if j == i:
                            continue
                        if xprime[j] == 0:
                            continue
            
                        if w[i] - w[j] <= res:
                            gain = q[i] - q[j] - (p[i][j] + p[j][i])
                            if gain > bgain:
                                bgain = gain
                                gaini = i
                                gainj = j
        if bgain == 0:
            break
    
        xprime[gaini] = 1
    
        if gainj != -1:
            xprime[gainj] = 0
            res += w[gainj] - w[gaini]
        else:
            res -= w[gaini]
    
        if res < 0:
            print("error\n")
            exit
                        
    gain = 0
    res = c
    for i in range(n):
        if xprime[i] == 0:
            continue
        res -= w[i]
        for j in range(n):
            if xprime[j] != 0:
                gain += p[i][j]

    if res < 0:
        print("error \n")
        exit 

    if gain > lb:
        lb = gain
        for i in range(n):
            xstar[i] = xprime[i]
    del(q)

    lb = 0
    for i in range(n):
        if xstar[i] != 0: 
            lb += p[i][i]
            for j in range(i+1,n):
                if xstar[j] != 0:
                    lb += p[i][j]

    return lb, xstar

In [59]:
def heuristica (dim, perc, id):
    
    n, p, w, c = read_instance(dim,perc,id)
    lbg, xp = guloso(n, p, w, c)
    lbm, xp = melhora(n, p, w, c, xp, lbg)

    #lower bound
    arquivo = open(f'result/qkp_heur_{dim}_{perc}.txt','a')
    arquivo.write(
        str(f"{dim}_{perc}_{id}")+';'
        +str(round(lbg,1))+';'
        +str(round(lbm,1))+'\n'
    )
    arquivo.close()

In [89]:
def qknapsack_linear(n, p, w, c):

    lbg, xp = guloso(n, p, w, c)
    lbm, xp = melhora(n, p, w, c, xp, lbg)

    #cria o modelo
    model = gp.Model("qkp_linear") 

    x = []
    for j in range(0, n):
        x.append(model.addVar(vtype=GRB.BINARY, name="x_{}".format(j+1)))

    l = list(tuple())
    for i in range(0, n):
        for j in range(i+1, n):
            l.append((i,j))

    y = model.addVars(l, vtype=GRB.BINARY, name='y')

    model.Params.TimeLimit = 120
    model.Params.MIPGap = 1.e-6
    model.Params.Threads = 1
    model.Params.Presolve = 0
    model.Params.Cuts = 0

    
    # Turn off display
    #gp.setParam('OutputFlag', 1)
    # Open log file
    logfile = open('heuristic.log', 'w')

    obj = 0 
    for i in range(0, n):
        obj += p[i][i] * x[i]
        for j in range(i+1, n):
            obj += p[i][j] * y[i,j]

    model.setObjective(obj, GRB.MAXIMIZE)

    constr = 0
    for j in range(0, n):
        constr += (w[j] * x[j])
    model.addConstr(constr <= c)

    for i in range(0,n):
        for j in range(i+1, n):
            constr1 = y[i,j]
            model.addConstr(constr1 <= x[i])

    for i in range(0,n):
        for j in range(i+1, n):
            constr2 = y[i,j]
            model.addConstr(constr2 <= x[j])

    for i in range(0,n):
        for j in range(i+1, n):
            constr3 = x[i] + x[j]
            model.addConstr(constr3 <= 1 + y[i,j])

    for i in range(n):
        x[i].Start = xp[i]


    #model.write(f"lp/qkp_linear_{dim}_{perc}_{t}.lp")

    model.optimize()
    
    status = 0
    if model.status == GRB.OPTIMAL:
        status = 1
 
    ub = model.objBound
    lb = model.objVal
    gap = model.MIPGap
    time = model.Runtime
    nodes = model.NodeCount

    model.dispose()

    # lower bound, upper bound, gap, time, nodes
    #arquivo = open(f'result/qkp_linear_{dim}_{perc}.txt','a')
    #arquivo.write(
    #    str(f"{dim}_{perc}_{t}")+';'
    #    +str(round(lb,1))+';'
    #    +str(round(ub,1))+';'
    #    +str(round(gap,2))+';'
    #    +str(round(time,2))+';'
    #    +str(round(nodes,1))+';'
    #    +str(round(status,1))+'\n'
    #)
    #arquivo.close()

    return lb, ub, gap, time, nodes, status

In [90]:
def qknapsack(dim,perc,t):

    n, p, w, c = read_instance(dim, perc, t)

    lb, ub, gap, time, nodes, status = qknapsack_linear(n, p, w, c)

    # lower bound, upper bound, gap, time, nodes
    arquivo = open(f'result/qkp_linear_{dim}_{perc}.txt','a')
    arquivo.write(
        str(f"{dim}_{perc}_{t}")+';'
        +str(round(lb,1))+';'
        +str(round(ub,1))+';'
        +str(round(gap,2))+';'
        +str(round(time,2))+';'
        +str(round(nodes,1))+';'
        +str(round(status,1))+'\n'
    )
    arquivo.close()

In [91]:
if __name__ == "__main__":

    dim = 50
    perc = 75
    id = 1 

    n, p, w, c = read_instance(dim,perc,id)

    #lbg, xp = guloso(n, p, w, c)
    #lbm, xp = melhora(n, p, w, c, xp, lbg)
    
    lb, ub, gap, time, nodes, status = qknapsack_linear(n, p, w, c)

    print("lb = ", lb)
    print("ub = ", ub)
    print("gap = ", gap)
    print("time = ", time)
    print("nodes = ", nodes)
    print("status = ", status)

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Set parameter Presolve to value 0
Set parameter Cuts to value 0


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 3676 rows, 1275 columns and 8625 nonzeros
Model fingerprint: 0x720e77a3
Variable types: 0 continuous, 1275 integer (1275 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 27.0000000
Variable types: 0 continuous, 1275 integer (1275 binary)
Found heuristic solution: objective 8185.0000000

Root relaxation: objective 1.441682e+04, 1232 iterations, 0.12 seconds (0.08 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 14416.8153    0  835 8185.00000 14416.8153  76.1%  

In [34]:
# __main__ : “top-level code environment”

# What is the “top-level code environment”?
# __main__ is the name of the environment where top-level code is run. 
# “Top-level code” is the first user-specified Python module that starts running. 
# It’s “top-level” because it imports all other modules that the program needs. 
# Sometimes “top-level code” is called an entry point to the application.

if __name__ == "__main__":

    for dim in [50]:
        for perc in [75]:
            for id in range(1,6):
                #instance = f"instances/{dim}/{dim}_{perc}_{id}.txt"
                heuristica(dim,perc,id)
                #lbg, x = guloso(n, p, w, c)
                #lbm, x = melhora(n, p, w, c, x, lbg)  